In [1]:
file_path = "../data/LesMiserables.txt"

In [10]:
with open(file_path, "r") as text_file:
    text = text_file.read()

In [5]:
text[0]

'The Project Gutenberg eBook of Les Misérables\n'

In [12]:
type(text)

str

In [13]:
text_paragraphs = text.split("\n\n")

In [14]:
text_paragraphs[0]

'The Project Gutenberg eBook of Les Misérables\n    \nThis ebook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever. You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this ebook or online\nat www.gutenberg.org. If you are not located in the United States,\nyou will have to check the laws of the country where you are located\nbefore using this eBook.'

In [15]:
len(text_paragraphs)

14558